# AFL Model - Part 4 - Weekly Predictions
Now that we have explored different algorithms for modelling, we can implement our chosen model and predict this week's AFL games! All you need to do is run the afl_modelling script each Thursday or Friday to predict the following week's games.

In [15]:
# Import Modules
from afl_feature_creation import prepare_afl_features
import afl_data_cleaning
import afl_feature_creation
import afl_modelling
import datetime
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

## Creating The Features For This Weekend's Games
To actually predict this weekend's games, we need to create the same features that we have created in the previous tutorials for the games that will be played this weekend. This includes all the rolling averages, efficiency features, elo features etc. So the majority of this tutorial will be using previously defined functions to create features for the following weekend's games.

### Create Next Week's DataFrame
Let's first get our cleaned afl_data dataset, as well as the odds for next weekend and the 2018 fixture.

In [16]:
# Grab the cleaned AFL dataset and the column order
afl_data = afl_data_cleaning.prepare_afl_data()
ordered_cols = afl_data.columns

# Define a function which grabs the odds for each game for the following weekend
def get_next_week_odds(path):
    # Get next week's odds
    next_week_odds = pd.read_csv(path)
    next_week_odds = next_week_odds.rename(columns={"team_1": "home_team", 
                                                "team_2": "away_team", 
                                                "team_1_odds": "odds", 
                                                "team_2_odds": "odds_away"
                                               })
    return next_week_odds

# Import the fixture
# Define a function which gets the fixture and cleans it up
def get_fixture(path):
    # Get the afl fixture
    fixture = pd.read_csv(path)

    # Replace team names and reformat
    fixture = fixture.replace({'Brisbane Lions': 'Brisbane', 'Footscray': 'Western Bulldogs'})
    fixture['Date'] = pd.to_datetime(fixture['Date']).dt.date.astype(str)
    fixture = fixture.rename(columns={"Home.Team": "home_team", "Away.Team": "away_team"})
    return fixture

next_week_odds = get_next_week_odds("data/weekly_odds.csv")
fixture = get_fixture("data/afl_fixture_2018.csv")

In [17]:
fixture.tail()

,Date,Season,Season.Game,Round,home_team,away_team,Venue
211,2018-08-24,2018,212,NaN,Brisbane,West Coast,Gabba
212,2018-08-24,2018,213,NaN,Carlton,St Kilda,Etihad Stadium
213,2018-08-24,2018,214,NaN,Richmond,Melbourne,MCG
214,2018-08-24,2018,215,NaN,Port Adelaide,Essendon,Adelaide Oval
215,2018-08-24,2018,216,NaN,Fremantle,Collingwood,Optus Stadium


Now that we have these DataFrames, we will define a function which combines the fixture and next week's odds to create a single DataFrame for the games over the next 7 days. To use this function we will need Game IDs for next week. So we will create another function which creates Game IDs by using the Game ID from the last game played and adding 1 to it.

In [18]:
# Define a function which creates game IDs for this week's footy games
def create_next_weeks_game_ids(afl_data):
    odds = get_next_week_odds("data/weekly_odds.csv")

    # Get last week's Game ID
    last_afl_data_game = afl_data['Game'].iloc[-1]

    # Create Game IDs for next week
    game_ids = [(i+1) + last_afl_data_game for i in range(odds.shape[0])]
    return game_ids

# Define a function which creates this week's footy game DataFrame
def get_next_week_df(afl_data):
    # Get the fixture and the odds for next week's footy games
    fixture = get_fixture("data/afl_fixture_2018.csv")
    next_week_odds = get_next_week_odds("data/weekly_odds.csv")
    next_week_odds['Game'] = create_next_weeks_game_ids(afl_data)

    # Get today's date and next week's date and create a DataFrame for next week's games
    todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
    date_in_7_days = (datetime.datetime.today() + datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    fixture = fixture[(fixture['Date'] >= todays_date) & (fixture['Date'] < date_in_7_days)].drop(columns=['Season', 'Season.Game', 'Round'])
    next_week_df = pd.merge(fixture, next_week_odds, on=['home_team', 'away_team'])

    # Split the DataFrame onto two rows for each game
    h_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds']]
    h_df['Team'] = h_df['home_team']
    h_df['Opposition'] = h_df['away_team']
    h_df['Home?'] = 1
    a_df = next_week_df[['Date', 'Game', 'home_team', 'away_team', 'odds_away']].rename(columns={'odds_away': 'odds'})
    a_df['Team'] = a_df['away_team']
    a_df['Opposition'] = a_df['home_team']
    a_df['Home?'] = 0
    next_week = a_df.append(h_df).sort_values(by='Game')
    return next_week

In [19]:
next_week_df = get_next_week_df(afl_data)
game_ids_next_round = create_next_weeks_game_ids(afl_data)
next_week_df.head()

,Date,Game,home_team,away_team,odds,Team,Opposition,Home?
0,2018-08-17,15369,Richmond,Essendon,2.88,Essendon,Richmond,0
0,2018-08-17,15369,Richmond,Essendon,1.53,Richmond,Essendon,1
1,2018-08-18,15370,Collingwood,Port Adelaide,2.92,Port Adelaide,Collingwood,0
1,2018-08-18,15370,Collingwood,Port Adelaide,1.54,Collingwood,Port Adelaide,1
2,2018-08-18,15371,Geelong,Fremantle,13.00,Fremantle,Geelong,0


### Create Each Feature
Now let's append next week's DataFrame to our afl_data DataFrame and then create all the features we used in the [AFL Feature Creation Tutorial](0.2. afl_feature_creation_tutorial.ipynb).

In [20]:
# Append next week's games to our afl_data DataFrame
afl_data = afl_data.append(next_week_df).reset_index(drop=True)
afl_data = afl_data[ordered_cols]

# Create disposal efficiency column
afl_data['disposal_efficiency'] = afl_data['ED'] / afl_data['D']

# Create rolling averages for our statistics
cols_indx_start = afl_data.columns.get_loc("GA")
afl_avgs = afl_feature_creation.create_rolling_averages(afl_data, 6, afl_data.columns[cols_indx_start:])

# Create form between teams feature
afl_avgs = afl_feature_creation.form_between_teams(afl_avgs, 6)

# Apply elos
elos, probs, elo_dict = afl_feature_creation.elo_applier(afl_data, 24)
afl_avgs['home_elo'] = afl_avgs['Game'].map(elos).str[0]
afl_avgs['away_elo'] = afl_avgs['Game'].map(elos).str[1]

# Get elos for next week
afl_avgs.loc[afl_avgs['home_elo'].isna(), 'home_elo'] = afl_avgs[afl_avgs['home_elo'].isna()]['home_team'].map(elo_dict)
afl_avgs.loc[afl_avgs['away_elo'].isna(), 'away_elo'] = afl_avgs[afl_avgs['away_elo'].isna()]['away_team'].map(elo_dict)

# Create Adjusted Margin and then Average it over a 6 game window
afl_avgs = afl_feature_creation.map_elos(afl_avgs)
afl_avgs['Adj_elo_ave_margin'] = afl_avgs['Margin'] * afl_avgs['elo_Opp'] / afl_avgs['elo']
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Adj_elo_ave_margin'])

# Create average elo of opponents beaten/lost to feature
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='beaten')
afl_avgs = afl_feature_creation.create_ave_elo_opponent(afl_avgs, 6, beaten_or_lost='lost')

# Create regular margin rolling average
afl_avgs = afl_feature_creation.create_rolling_averages(afl_avgs, 6, ['Margin'])
afl_avgs.tail()

,Team,odds,Date,home_team,away_team,Game,Round,Goals,Behinds,Points,Venue,Home?,Opposition,Opposition Goals,Opposition Behinds,Opposition Points,Season,Status,GA_ave_6,CP_ave_6,UP_ave_6,ED_ave_6,CM_ave_6,MI5_ave_6,One.Percenters_ave_6,BO_ave_6,K_ave_6,HB_ave_6,D_ave_6,M_ave_6,G_ave_6,B_ave_6,T_ave_6,HO_ave_6,I50_ave_6,CL_ave_6,CG_ave_6,R50_ave_6,FF_ave_6,FA_ave_6,AF_ave_6,SC_ave_6,CCL_ave_6,SCL_ave_6,SI_ave_6,MG_ave_6,TO_ave_6,ITC_ave_6,T5_ave_6,disposal_efficiency_ave_6,form_over_opposition_6,home_elo,away_elo,elo,elo_Opp,Adj_elo_ave_margin_ave_6,average_elo_opponents_beaten_6,average_elo_opponents_lost_6,Margin_ave_6
3015,Carlton,4.80,2018-08-19,Carlton,Western Bulldogs,15375,NaN,NaN,NaN,NaN,NaN,1,Western Bulldogs,NaN,NaN,NaN,NaN,NaN,6.000000,131.333333,224.833333,260.500000,11.500000,9.333333,43.166667,6.833333,196.166667,162.166667,358.333333,85.666667,8.666667,6.333333,59.333333,34.500000,41.500000,33.666667,55.666667,38.333333,20.000000,22.333333,1453.000000,1530.666667,12.000000,21.666667,76.166667,5001.500000,65.333333,62.333333,7.666667,0.728181,2,1254.391944,1436.609139,1254.391944,1436.609139,-40.274325,1497.756493,1459.288681,-34.500000
3016,Melbourne,2.28,2018-08-19,West Coast,Melbourne,15376,NaN,NaN,NaN,NaN,NaN,0,West Coast,NaN,NaN,NaN,NaN,NaN,10.666667,168.500000,232.000000,279.166667,11.166667,12.000000,50.333333,2.833333,216.166667,183.666667,399.833333,84.500000,15.000000,10.666667,74.833333,54.333333,62.333333,43.166667,59.833333,30.500000,19.166667,20.833333,1680.333333,1714.500000,14.833333,28.333333,117.666667,6266.833333,75.166667,78.833333,14.500000,0.694229,1,1591.711462,1509.390579,1509.390579,1591.711462,34.258681,1490.159807,1633.148523,32.666667
3017,West Coast,1.76,2018-08-19,West Coast,Melbourne,15376,NaN,NaN,NaN,NaN,NaN,1,Melbourne,NaN,NaN,NaN,NaN,NaN,7.166667,135.833333,210.333333,255.333333,12.833333,10.166667,48.666667,3.166667,222.166667,125.166667,347.333333,98.333333,10.000000,9.000000,67.000000,41.666667,51.500000,33.500000,50.500000,38.500000,24.166667,20.333333,1553.666667,1609.833333,9.833333,23.666667,79.166667,5513.000000,71.500000,72.833333,9.333333,0.733040,5,1591.711462,1509.390579,1591.711462,1509.390579,-0.850816,1529.169608,1594.198154,-0.666667
3018,North Melbourne,3.20,2018-08-19,Adelaide,North Melbourne,15377,NaN,NaN,NaN,NaN,NaN,0,Adelaide,NaN,NaN,NaN,NaN,NaN,9.333333,154.833333,216.833333,259.333333,11.833333,11.333333,49.333333,4.000000,191.666667,175.500000,367.166667,77.166667,12.833333,7.666667,63.500000,44.833333,52.833333,37.833333,55.500000,34.666667,20.833333,18.000000,1507.833333,1634.833333,14.166667,23.666667,89.500000,5442.166667,71.333333,73.666667,13.666667,0.704759,3,1620.104725,1472.910710,1472.910710,1620.104725,7.981990,1570.161952,1510.560437,6.833333
3019,Adelaide,1.46,2018-08-19,Adelaide,North Melbourne,15377,NaN,NaN,NaN,NaN,NaN,1,North Melbourne,NaN,NaN,NaN,NaN,NaN,5.166667,155.000000,193.666667,238.666667,10.166667,7.833333,46.000000,3.333333,220.000000,133.666667,353.666667,80.333333,9.333333,9.166667,72.000000,39.500000,50.166667,38.333333,49.833333,35.500000,19.166667,18.666667,1524.166667,1581.666667,10.166667,28.166667,69.500000,5638.833333,71.666667,76.000000,13.166667,0.673436,3,1620.104725,1472.910710,1620.104725,1472.910710,-16.628460,1446.170730,1491.105820,-18.000000


Our DataFrame looks great! Although there are some NaNs, these are only in columns like 'Goals' which obviously don't have a value yet as the game hasn't been played.

Now we need to get our DataFrame on one line so that each row corresponds to one footy game. Let's use our previously defined function for this. Let's also drop the columns with NaNs which we don't need.

In [21]:
# Get each footy match on individual rows and drop irrelevant columns
one_line = afl_modelling.get_df_on_one_line(afl_avgs)

# Drop duplicate columns and unnecessary columns
cols_to_drop = ['Opposition Behinds', 'Goals', 'Behinds', 'Opposition Goals', 'Opposition Points', 'Points', 'Round', 'Venue', 'Season', 'Status',
               'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'elo', 'elo_Opp', 'Behinds_away',
               'Goals_away', 'home_elo_away', 'away_elo_away', 'elo_away', 'elo_Opp_away']
one_line = one_line.drop(columns=cols_to_drop)
# Drop all columns where home_elo or away_elo == 1500 exactly, as this is the first game played
one_line = one_line[(one_line['home_elo'] != 1500) & (one_line['away_elo'] != 1500)]

# Drop Na rows from calculating moving averages
one_line = one_line.dropna(axis=0)

Now let's create our differential_df and then filter out DataFrame to only include this weekend's game, based on the Game IDs we created earlier

In [22]:
diff_df = afl_modelling.get_diff_df(one_line)
diff_df = diff_df.drop(columns=['odds', 'odds_away']).select_dtypes(include=[np.number])
prediction_feature_set = diff_df[diff_df['Game'].isin(game_ids_next_round)].dropna(axis=1)

In [23]:
prediction_feature_set

,Game,home_elo,away_elo,CCL_ave_6_away,SCL_ave_6_away,SI_ave_6_away,MG_ave_6_away,TO_ave_6_away,ITC_ave_6_away,T5_ave_6_away,GA_ave_6_diff,CP_ave_6_diff,UP_ave_6_diff,ED_ave_6_diff,CM_ave_6_diff,MI5_ave_6_diff,One.Percenters_ave_6_diff,BO_ave_6_diff,K_ave_6_diff,HB_ave_6_diff,D_ave_6_diff,M_ave_6_diff,G_ave_6_diff,B_ave_6_diff,T_ave_6_diff,HO_ave_6_diff,I50_ave_6_diff,CL_ave_6_diff,CG_ave_6_diff,R50_ave_6_diff,FF_ave_6_diff,FA_ave_6_diff,AF_ave_6_diff,SC_ave_6_diff,disposal_efficiency_ave_6_diff,form_over_opposition_6_diff,Adj_elo_ave_margin_ave_6_diff,average_elo_opponents_beaten_6_diff,average_elo_opponents_lost_6_diff,Margin_ave_6_diff,implied_odds_prob,implied_odds_prob_away
1531,15369,1624.637563,1518.535623,13.666667,26.000000,109.000000,5962.666667,70.166667,73.166667,12.333333,-2.000000,-3.500000,0.500000,-2.500000,0.000000,-0.666667,3.166667,-4.666667,-15.333333,8.166667,-7.166667,-2.666667,0.666667,-1.833333,-7.666667,-11.833333,1.166667,-7.333333,0.166667,-0.333333,-2.666667,0.666667,-82.666667,-52.500000,0.007103,4,-5.640472,-20.039832,80.593175,-3.833333,0.653595,0.347222
1532,15370,1501.449238,1563.944697,10.000000,27.166667,81.000000,5807.333333,75.666667,78.166667,16.666667,1.666667,-4.166667,42.666667,52.833333,0.166667,0.000000,-7.500000,1.000000,-2.833333,37.833333,35.000000,8.833333,2.833333,-0.333333,-15.500000,0.000000,-0.666667,-0.333333,2.333333,-3.000000,-0.166667,-1.500000,52.666667,36.666667,0.072608,-4,2.327358,3.192648,61.292394,2.833333,0.649351,0.342466
1533,15371,1643.524765,1426.405257,10.000000,22.166667,80.833333,5296.000000,74.500000,70.666667,10.500000,3.166667,9.666667,15.500000,24.666667,3.333333,1.500000,3.666667,0.000000,5.500000,16.666667,22.166667,5.666667,3.333333,1.666667,-3.000000,-3.833333,8.166667,6.666667,1.833333,-0.333333,2.333333,2.333333,68.000000,60.500000,0.020977,2,21.027604,-17.558351,13.625954,20.333333,0.925926,0.076923
1534,15372,1613.612204,1630.736013,10.333333,25.333333,70.666667,5804.333333,73.000000,70.166667,8.500000,1.666667,0.166667,8.500000,8.166667,0.333333,1.166667,10.833333,1.666667,17.333333,-9.666667,7.666667,6.333333,1.166667,2.666667,-3.333333,12.333333,7.166667,7.500000,-4.833333,-4.333333,0.833333,-4.166667,73.666667,57.333333,0.007920,0,15.937544,-47.541945,67.422697,17.666667,0.552486,0.454545
1535,15373,1416.739182,1600.259805,13.333333,20.833333,108.666667,5658.500000,70.500000,71.666667,12.000000,-3.000000,-1.166667,2.000000,-12.833333,-1.833333,-2.500000,-7.666667,-0.666667,-9.333333,7.666667,-1.666667,-0.666667,-4.500000,0.833333,-7.166667,-4.000000,-0.833333,3.500000,-1.666667,3.166667,1.333333,-1.166667,-68.666667,-46.666667,-0.030473,-4,-39.514193,-84.599760,134.741125,-38.500000,0.169492,0.833333
1536,15374,1294.817584,1280.219510,12.166667,27.500000,101.333333,5755.000000,65.666667,63.833333,9.000000,-6.333333,6.166667,-57.000000,-69.166667,0.333333,-5.000000,-2.333333,0.500000,-18.000000,-32.833333,-50.833333,-24.833333,-4.666667,4.000000,15.666667,0.500000,-4.166667,-3.000000,3.666667,0.000000,-3.333333,1.500000,-162.833333,-197.333333,-0.091850,0,-25.860074,-34.587977,-100.990452,-25.833333,0.208333,0.787402
1537,15375,1254.391944,1436.609139,9.666667,26.666667,76.500000,5215.000000,74.000000,69.500000,11.666667,-0.333333,-6.166667,-20.833333,-21.833333,4.000000,-1.000000,-10.000000,0.333333,-7.000000,-17.833333,-24.833333,-0.166667,-1.333333,-1.166667,-5.333333,2.833333,-10.166667,-2.666667,2.333333,5.166667,0.166667,2.333333,-91.666667,-78.333333,-0.007319,-2,-9.124741,29.150273,-83.152419,-5.500000,0.208333,0.793651
1538,15376,1591.711462,1509.390579,14.833333,28.333333,117.666667,6266.833333,75.166667,78.833333,14.500000,-3.500000,-32.666667,-21.666667,-23.833333,1.666667,-1.833333,-1.666667,0.333333,6.000000,-58.500000,-52.500000,13.833333,-5.000000,-1.666667,-7.833333,-12.666667,-10.833333,-9.666667,-9.333333,8.000000,5.000000,-0.500000,-126.666667,-104.666667,0.038811,4,-35.109497,39.009802,-38.950369,-33.333333,0.568182,0.43859

Great! We now have our feature DataFrame for this weekend's games. Let's now model this to create our predictions

## Creating Our Predictions
To create our predictions we will use similar code to what we used in our AFL Modelling Tutorial. We will need to train our model on all the data we have (up until last week's games), and then use our trained model to predict this week. Again, we will use Stacking with XGB, like we did in our AFL Modelling Tutorial.

In [24]:
afl = prepare_afl_features(window=6, k_factor=24).dropna().sort_values(by='Game')
afl = afl_modelling.get_df_on_one_line(afl)

# Drop columns which leak data and which we don't need
dropped_cols = ['Behinds', 'Goals', 'Opposition Behinds', 'Opposition Goals', 'Opposition Points', 'Points',
               'elo', 'elo_Opp', 'CCL_ave_6', 'SCL_ave_6', 'SI_ave_6', 'MG_ave_6', 'TO_ave_6', 'ITC_ave_6', 'T5_ave_6', 'home_win_away',
               'home_elo_away', 'away_elo_away']
afl = afl.drop(columns=dropped_cols)
# Get a differential DataFrame - subtracting the away features from the home features
diff_df = afl_modelling.get_diff_df(afl)

In [25]:
# Hard code estimators from our last tutorial
all_estimators = [
    LogisticRegression(C=0.75, class_weight=None, dual=False, fit_intercept=True,
       intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
       penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
       verbose=0, warm_start=False),
    
    RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
     max_iter=None, normalize=False, random_state=None, solver='auto',
     tol=0.001),
    
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
         max_depth=15, max_features='sqrt', max_leaf_nodes=None,
         min_impurity_decrease=0.0, min_impurity_split=None,
         min_samples_leaf=1, min_samples_split=10,
         min_weight_fraction_leaf=0.0, n_estimators=750, n_jobs=-1,
         oob_score=False, random_state=5, verbose=0, warm_start=False),
    
    GaussianNB(priors=None),
    
    LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
           solver='svd', store_covariance=False, tol=0.0001)
]

# Hard code best cols
best_cols = ['home_elo',
'away_elo',
'GA_ave_6_diff',
'CP_ave_6_diff',
'UP_ave_6_diff',
'ED_ave_6_diff',
'CM_ave_6_diff',
'MI5_ave_6_diff',
'One.Percenters_ave_6_diff',
'BO_ave_6_diff',
'HB_ave_6_diff',
'M_ave_6_diff',
'G_ave_6_diff',
'T_ave_6_diff',
'HO_ave_6_diff',
'I50_ave_6_diff',
'CL_ave_6_diff',
'CG_ave_6_diff',
'R50_ave_6_diff',
'FF_ave_6_diff',
'FA_ave_6_diff',
'AF_ave_6_diff',
'SC_ave_6_diff',
'disposal_efficiency_ave_6_diff',
'R50_efficiency_ave_6_diff',
'I50_efficiency_ave_6_diff',
'Adj_elo_ave_margin_ave_6_diff',
'average_elo_opponents_beaten_6_diff',
'average_elo_opponents_lost_6_diff',
'Margin_ave_6_diff',
'implied_odds_prob',
'implied_odds_prob_away']

In [26]:
# Create our train sets
X = diff_df.drop(columns=['home_win']).select_dtypes(include=[np.number])
y = diff_df['home_win']

features = prediction_feature_set.columns

# Predict Next Round
preds_next_round = afl_modelling.implement_xgb_stacking(X[features].drop(columns='Game'), y, 
                                                        prediction_feature_set.drop(columns='Game'), all_estimators)

C:\Users\wardj\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Now that we have our predictions, the final step is to put our predictions into a DataFrame so they're easy on the eye, and our predictions correspond to actual home and away teams rather than Game IDs. This is the final step.

In [27]:
preds_df = pd.DataFrame({
    "Game": prediction_feature_set['Game'],
    "Prediction (home_win)": preds_next_round
})

prediction_feature_set['home_odds'] = 1 / prediction_feature_set['implied_odds_prob']
prediction_feature_set['away_odds'] = 1 / prediction_feature_set['implied_odds_prob_away']

final_preds_df = pd.merge(preds_df, next_week_df[['home_team', 'away_team', 'Game']], on='Game').drop_duplicates()
final_preds_df = pd.merge(final_preds_df, prediction_feature_set[['home_odds', 'away_odds', 'home_elo', 'away_elo', 'Game']], on='Game')
final_preds_df['Predicted Winner'] = final_preds_df.apply(lambda x: x['home_team'] if x['Prediction (home_win)'] == 1 else x['away_team'], axis=1)

In [31]:
final_preds_df[['Game', 'home_team', 'away_team', 'Predicted Winner', 'home_odds', 'away_odds', 'home_elo', 'away_elo']]

,Game,home_team,away_team,Predicted Winner,home_odds,away_odds,home_elo,away_elo
0,15369,Richmond,Essendon,Richmond,1.53,2.88,1624.637563,1518.535623
1,15370,Collingwood,Port Adelaide,Collingwood,1.54,2.92,1501.449238,1563.944697
2,15371,Geelong,Fremantle,Geelong,1.08,13.00,1643.524765,1426.405257
3,15372,GWS,Sydney,GWS,1.81,2.20,1613.612204,1630.736013
4,15373,St Kilda,Hawthorn,Hawthorn,5.90,1.20,1416.739182,1600.259805
5,15374,Gold Coast,Brisbane,Brisbane,4.80,1.27,1294.817584,1280.219510
6,15375,Carlton,Western Bulldogs,Carlton,4.80,1.26,1254.391944,1436.609139
7,15376,West Coast,Melbourne,Melbourne,1.76,2.28,1591.711462,1509.390579
8,15377,Adelaide,North Melbourne,Adelaide,1.46,3.20,1620.104725,1472.910710


## Conclusion
Congratulations! You have created AFL predictions for this week. If you are beginner to this, don't be overwhelmed. The process gets easier each time you do it. And it is super rewarding. In future iterations we will update this tutorial to predict actual odds, and then integrate this with Betfair's API so that you can create an automated betting strategy using Machine Learning to create your predictions!